In [1]:
# automatically upload modules
%load_ext autoreload
%autoreload 2

In [2]:
from argparse import Namespace
import os
import pandas as pd
import pickle
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import torch
import pickle

from genome_embeddings import data_viz
from genome_embeddings import evaluate
from genome_embeddings import models
from genome_embeddings import train_test
from genome_embeddings import util
from genome_embeddings import trainable # import before ray (?)
import ray

In [3]:
flags = Namespace(
    DATA_FP = '/Users/natasha/Desktop/mcgill_postdoc/ncbi_genomes/genome_embeddings/data/', 
    SAVE_FP = '/Users/natasha/Desktop/mcgill_postdoc/ncbi_genomes/genome_embeddings/',
    KEGG_FP = '/Users/natasha/Desktop/mcgill_postdoc/ncbi_genomes/kegg_dataset/',
    data_source = 'kegg', #['get_homologues' | 'kegg']
    n_test = 0.1, # train-test split, n * 100 = % of data that goes into test set (e.g.: 0.1 -> 10%)
    num_epochs = 1, # IF CIRRICULUM, DO NOT SET < 3
    batch_size = 128,
    lr = 1e-3,
    print_every = 50, # print loss every n batches during training (5)
    replacement_threshold = 0.5, # probability over which binarizer converts to a 1
    num_corruptions = 100, # number of corrupted versions of a genome to produce
    corruption_fraction = 0.5, # fraction of genes to retain during corruption process
    phy_mode = "bacteria_only", # training with only bacteria vs also euk/arch
    cirriculum = False, # implement cirriculum learning based on gene count
    rare_threshold = 10, # drop features that occur fewer than this times in training ds 
    weight_decay=0.1 # L2 regularization
    )

In [4]:
#(lambda spec: 10**(-10 * np.random.rand()))(1)

### Data exploration + preprocessing 

In [5]:
# First create genome representations (very slow)
# Each genome is a list of KO's and/or KEGG modules
if os.path.isfile(flags.DATA_FP+'genome_to_mod.csv'):
    print("Genome representations already exist")
else:
    genome_rep.genome_kos(flags.KEGG_FP)
    print("Must generate genome representations from scratch. This will take several hours.")

Genome representations already exist


In [6]:
df, cluster_names = util.load_data(flags.DATA_FP, flags.data_source)
genome_to_tax = util.genome_to_tax(df)
with open('cluster_names.txt', 'wb') as filehandle:
    pickle.dump(cluster_names, filehandle)


In [7]:
#data_viz.tax_distrib(df, genome_to_tax)

In [8]:
#data_viz.module_stats(df)

In [9]:
#data_viz.genes_per_genome(df)

In [10]:
# genome_to_tax2 = {}
# for i in df.index:
#     genome_to_tax2[i] = genome_to_tax[i]
    
# # Create dict mapping each genome to a unique numerical ID
# genome_to_num ={}
# for i,genome in enumerate(df.index):
#     genome_to_num[genome] = i
    
# num_to_genome = {v: k for k, v in genome_to_num.items()}

In [11]:
# Split train-test sets in a phylogenetically balanced manner 
if os.path.isfile(flags.DATA_FP+'uncorrupted_train_balanced.csv'):
    print("Train-test split already exists, loading from file")
    train_orig = pd.read_csv(flags.DATA_FP+"uncorrupted_train_balanced.csv", index_col=0)    
    test_orig = pd.read_csv(flags.DATA_FP+"uncorrupted_test_balanced.csv", index_col=0)    

else:
        
#     genome_to_tax2 = {}

#     for i in df.index:
#         genome_to_tax2[i] = genome_to_tax[i]

    # Create dict mapping each genome to a unique numerical ID
    genome_to_num ={}
    for i,genome in enumerate(df.index):
        genome_to_num[genome] = i

    num_to_genome = {v: k for k, v in genome_to_num.items()}
        
    print("Generating train-test split")
    train_orig, test_orig = util.balanced_split(df, flags.n_test, genome_to_tax, 
                                                num_to_genome, flags.DATA_FP)    
    train_orig.to_csv(flags.DATA_FP+'uncorrupted_train_balanced.csv')
    test_orig.to_csv(flags.DATA_FP+'uncorrupted_test_balanced.csv')

Train-test split already exists, loading from file


In [12]:
#data_viz.hist_prob_ko(train_orig)

In [13]:
if flags.phy_mode == "bacteria_only":
    train_genomes = train_orig.index.to_list()
    test_genomes = test_orig.index.to_list()
    
    unf_train_data, train_tax_dict = util.bacteria_only(train_orig, train_genomes, genome_to_tax)
    unf_test_data, test_tax_dict = util.bacteria_only(test_orig, test_genomes, genome_to_tax)

In [14]:
# Remove rare features from train + test datasets
# Rare = fewer than n occurences in training dataset
# Last argument specifies n, set to correspond to 1% of genomes (3432 genomes -> n = 34)
train_data, test_data, cluster_names = util.remove_rare(unf_train_data, unf_test_data, 
                                                        cluster_names, unf_train_data.shape[0]*0.01)

In [15]:
# Produce corrupted genomes
# Could eventually do re-sampling / extra-corrupting to have more examples of "rare" genome types
#    e.g.: those from underrepresented groups M00003   

if os.path.isfile(flags.DATA_FP+'corrupted_train.pt'):
    print("Corrupted genomes already exist")
#    train_data = torch.load(flags.DATA_FP+"corrupted_train.pt")
#    test_data = torch.load(flags.DATA_FP+"corrupted_test.pt")
#    genome_idx_train = torch.load(flags.DATA_FP+"genome_idx_train.pt")
#    genome_idx_test = torch.load(flags.DATA_FP+"genome_idx_test.pt")
else:
    print("Generating corrupted dataset from scratch with",flags.num_corruptions,"corruptions per genome")
    train_data, genome_idx_train = util.corrupt(train_data, flags.num_corruptions, flags.corruption_fraction, 
                                                cluster_names, "train", flags.DATA_FP)

    test_data, genome_idx_test = util.corrupt(test_data, flags.num_corruptions, flags.corruption_fraction, 
                                              cluster_names, "test", flags.DATA_FP)

Corrupted genomes already exist


In [16]:
# print(("There are %s genomes and %s features in the training dataset") % 
#       (train_data.shape[0],int(train_data.shape[1]/2)))

# print(("There are %s genomes and %s features in the test dataset") % 
#       (test_data.shape[0],int(test_data.shape[1]/2)))

In [17]:
# if flags.cirriculum:
#     loaders = util.cirriculum_load(train_data, test_data, flags.batch_size, 
#                            flags.batch_size, cluster_names)
# else:
#     loaders = util.dataloaders(train_data, test_data, flags.batch_size, 
#                                flags.batch_size, cluster_names)

### Define and train network

In [18]:
train_data.shape[1]

7065

In [19]:
#num_features = train_data.dataset.shape[1]
num_features = train_data.shape[1] # Number of features in the entire dataset (train + test)

In [20]:
# define the network
model = models.AutoEncoder(num_features)
torch.save(model, flags.DATA_FP+"model.pt")
print(model)

AutoEncoder(
  (e1): Linear(in_features=7065, out_features=3532, bias=True)
  (e2): Linear(in_features=3532, out_features=1766, bias=True)
  (e3): Linear(in_features=1766, out_features=1177, bias=True)
  (e4): Linear(in_features=1177, out_features=883, bias=True)
  (e5): Linear(in_features=883, out_features=706, bias=True)
  (lv): Linear(in_features=706, out_features=588, bias=True)
  (d1): Linear(in_features=588, out_features=706, bias=True)
  (d2): Linear(in_features=706, out_features=883, bias=True)
  (d3): Linear(in_features=883, out_features=1177, bias=True)
  (d4): Linear(in_features=1177, out_features=1766, bias=True)
  (d5): Linear(in_features=1766, out_features=3532, bias=True)
  (output_layer): Linear(in_features=3532, out_features=7065, bias=True)
)


In [38]:
train_vars = {"batch_size": tune.grid_search([32, 64]), # [32, 64, 128, 256]
              "num_epochs": flags.num_epochs,
              "replacement_threshold": flags.replacement_threshold,
              "lr": tune.grid_search([0.1, 0.0001]),
              "weight_decay": tune.grid_search([0.01, 0.00001]) 
}

In [39]:
ray.shutdown()
ray.init(local_mode=True)

2020-07-03 16:40:41,434	INFO resource_spec.py:212 -- Starting Ray with 10.84 GiB memory available for workers and up to 5.44 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-03 16:40:41,727	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.2.23',
 'raylet_ip_address': '192.168.2.23',
 'redis_address': '192.168.2.23:32122',
 'object_store_address': '/tmp/ray/session_2020-07-03_16-40-41_413523_84062/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-03_16-40-41_413523_84062/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-07-03_16-40-41_413523_84062'}

In [40]:
analysis = tune.run(
    trainable.train_AE, 
    name="exp_1",
    config=train_vars, 
    verbose=2, 
    resources_per_trial={
            "cpu": 2,
            "gpu": 0
    },
    num_samples=1)

print("Best config is:", analysis.get_best_config(metric="mean_accuracy"))

2020-07-03 16:40:42,035	INFO trainable.py:217 -- Getting current IP.


batch_size 32
time to load data from file 5.245208740234375e-06
time to create dataloaders 0.0001270771026611328
done training


2020-07-03 16:40:47,725	WARNING util.py:137 -- The `start_trial` operation took 5.7159740924835205 seconds to complete, which may be a performance bottleneck.


f1 0.6040893026509886


Trial name,status,loc,batch_size,lr,weight_decay
train_AE_00000,RUNNING,,32,0.1,0.01
train_AE_00001,PENDING,,64,0.1,0.01
train_AE_00002,PENDING,,32,0.0001,0.01
train_AE_00003,PENDING,,64,0.0001,0.01
train_AE_00004,PENDING,,32,0.1,1e-05
train_AE_00005,PENDING,,64,0.1,1e-05
train_AE_00006,PENDING,,32,0.0001,1e-05
train_AE_00007,PENDING,,64,0.0001,1e-05


...

f1 0.4759355833485823


Trial name,status,loc,batch_size,lr,weight_decay,acc,iter,total time (s)
train_AE_00000,RUNNING,,32,0.1,0.01,,,
train_AE_00001,RUNNING,,64,0.1,0.01,,,
train_AE_00002,TERMINATED,,32,0.0001,0.01,0.47317,1,3.90561
train_AE_00003,TERMINATED,,64,0.0001,0.01,0.471649,1,4.89822
train_AE_00004,RUNNING,,32,0.1,1e-05,,,
train_AE_00005,RUNNING,,64,0.1,1e-05,,,
train_AE_00006,RUNNING,,32,0.0001,1e-05,,,
train_AE_00007,RUNNING,,64,0.0001,1e-05,,,


Result for train_AE_00004:
  date: 2020-07-03_16-41-05
  done: false
  experiment_id: fb8f3b503b154937b746ab62d0eecef4
  experiment_tag: 4_batch_size=32,lr=0.1,weight_decay=1e-05
  hostname: natashas-MacBook-Pro.local
  iterations_since_restore: 1
  mean_accuracy: 0.605495239240283
  node_ip: 192.168.2.23
  pid: 84062
  time_since_restore: 3.783276081085205
  time_this_iter_s: 3.783276081085205
  time_total_s: 3.783276081085205
  timestamp: 1593808865
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '00004'
  
Result for train_AE_00005:
  date: 2020-07-03_16-41-10
  done: false
  experiment_id: a1aa4e5fe6774d399f2397653d2e8a58
  experiment_tag: 5_batch_size=64,lr=0.1,weight_decay=1e-05
  hostname: natashas-MacBook-Pro.local
  iterations_since_restore: 1
  mean_accuracy: 0.5984394735203908
  node_ip: 192.168.2.23
  pid: 84062
  time_since_restore: 4.668767213821411
  time_this_iter_s: 4.668767213821411
  time_total_s: 4.668767213821411
  timestamp: 1593808870
  timesteps

Trial name,status,loc,batch_size,lr,weight_decay,acc,iter,total time (s)
train_AE_00000,TERMINATED,,32,0.1,0.01,0.604089,1,5.68551
train_AE_00001,TERMINATED,,64,0.1,0.01,0.580173,1,5.40641
train_AE_00002,TERMINATED,,32,0.0001,0.01,0.47317,1,3.90561
train_AE_00003,TERMINATED,,64,0.0001,0.01,0.471649,1,4.89822
train_AE_00004,TERMINATED,,32,0.1,1e-05,0.605495,1,3.78328
train_AE_00005,TERMINATED,,64,0.1,1e-05,0.598439,1,4.66877
train_AE_00006,TERMINATED,,32,0.0001,1e-05,0.467623,1,3.50392
train_AE_00007,TERMINATED,,64,0.0001,1e-05,0.475936,1,4.26014


Best config is: {'batch_size': 32, 'num_epochs': 1, 'replacement_threshold': 0.5, 'lr': 0.1, 'weight_decay': 1e-05}


In [ ]:
print("Best batch size:", analysis.get_best_config(metric="mean_accuracy")["batch_size"])

In [ ]:
# train the model
# train_losses, test_losses, train_f1_scores, test_f1_scores = train_test.train_model(loaders, 
#         model, flags.num_epochs, flags.print_every,
#         flags.SAVE_FP, flags.replacement_threshold, cluster_names, flags.cirriculum, train_data[:,:len(cluster_names)],
#         search_space)
#train_losses, test_losses, train_f1_scores, test_f1_scores = train_test.train_model(train_vars, hyperparams)

### Evaluate model performance

In [ ]:
# evaluate model performance
perf_lc = data_viz.learning_curve(train_f1_scores, test_f1_scores, "performance", flags.cirriculum)

In [ ]:
# evaluate model performance
optim_lc = data_viz.learning_curve(train_losses, test_losses, "optimization", flags.cirriculum)

In [ ]:
# first convert test_data from subset -> tensor, split corrupt vs target sets
tensor_test_data = torch.tensor([i.numpy() for i in test_data]).float()
corrupt_test_data = tensor_test_data[:,:len(cluster_names)]
target = tensor_test_data[:,len(cluster_names):].detach().numpy()

In [ ]:
# Generate probabilities for ROC curve
model.eval()
with torch.no_grad():
    y_probas = model(corrupt_test_data) # predicted probabilities generated by model

In [ ]:
roc = data_viz.my_roc_curve(target, y_probas.numpy())

In [ ]:
util.log_results(roc, optim_lc, perf_lc, flags, model)

In [ ]:
# create embeddings for test set
#uncorrupt_test_data = tensor_test_data[:,len(cluster_names):]
#tensor_test_data = torch.tensor([i.numpy() for i in test_data]).float()
embeddings = train_test.generate_embeddings(model, corrupt_test_data)

In [ ]:
#data_viz.plot_tSNE(embeddings, test_data, num_to_genome, genome_to_tax, test_tax_dict)

In [ ]:
# tSNE for corrupted genomes passed through untrained model
untrained_model = models.AutoEncoder(len(cluster_names))
untr_embeddings = train_test.generate_embeddings(untrained_model, corrupt_test_data)

In [ ]:
#data_viz.plot_tSNE(untr_embeddings, test_data, num_to_genome, genome_to_tax, test_tax_dict)
# data_viz.plot_tSNE(untr_embeddings, test_data, num_to_genome, genome_to_tax, genome_idx_test)

In [ ]:
# Evaluate model and compare against baselines
# Get corrupted input set, target set, and predictions set (binarized to 1's and 0's)
#corrupt_test_data = tensor_test_data[:,:len(cluster_names)]

model.eval()
with torch.no_grad():
    pred = model.forward(corrupt_test_data).detach().numpy()
b_pred = train_test.binarize(pred, flags.replacement_threshold)

In [ ]:
# Generate confusion matrix
cm = evaluate.dom_confusion_matrix(b_pred, target, num_to_genome, genome_to_tax, test_tax_dict, genome_idx_test)

In [ ]:
util.log_results(roc, optim_lc, perf_lc, flags, model, cm)

In [ ]:
# Baseline 1: untrained DAE
# Generate predictions using an untrained DAE model
model.eval()
with torch.no_grad():
    untr_pred = untrained_model.forward(corrupt_test_data).detach().numpy()
untr_b_preds = train_test.binarize(untr_pred, flags.replacement_threshold)

In [ ]:
# if os.path.isfile(flags.DATA_FP+"rand_b_pred.pt"):
#     print("Loading random predictions from file")
#     rand_b_pred = torch.load(flags.DATA_FP+"rand_b_pred.pt")
# else: 
#     # This is slow
#     print("Generating random predictions, this will take a while (~30 min)")
#     rand_b_pred = evaluate.generate_baseline(num_features, train_data, 
#                                              corrupt_test_data, "base_random", cluster_names)
#     torch.save(rand_b_pred, flags.DATA_FP+"rand_b_pred.pt")

rand_b_pred = evaluate.generate_baseline(num_features, train_data, 
                                         corrupt_test_data, "base_random", cluster_names)

In [ ]:
torch.save(rand_b_pred, flags.DATA_FP+"rand_b_pred.pt")

In [ ]:
# if os.path.isfile(flags.DATA_FP+"smart_b_pred.pt"):
#     print("Loading smart random predictions from file")
#     smart_b_pred = torch.load(flags.DATA_FP+"smart_b_pred.pt")
# else:
#     print("Generating smart random predictions, this will take a while (~30 min)")
#     smart_b_pred = evaluate.generate_baseline(num_features, train_data, 
#                                           corrupt_test_data, "smart_random", cluster_names)
#     torch.save(smart_b_pred, flags.DATA_FP+"smart_b_pred.pt")

smart_b_pred = evaluate.generate_baseline(num_features, train_data, 
                                      corrupt_test_data, "smart_random", cluster_names)

In [ ]:
torch.save(smart_b_pred, flags.DATA_FP+"smart_b_pred.pt")

In [ ]:
import numpy as np
np.sum(smart_b_pred == rand_b_pred), np.sum(smart_b_pred != rand_b_pred)

In [ ]:
import pandas as pd
hs = evaluate.hamming(target, b_pred)
hs_stats = [round(sum(hs)/len(hs),2), round(min(hs),2), round(max(hs),2)]

untr_hs = evaluate.hamming(target, untr_b_preds)
untr_hs_stats = [round(sum(untr_hs)/len(untr_hs),2), round(min(untr_hs),2), round(max(untr_hs),2)]

rand_hs = evaluate.hamming(target, rand_b_pred)
rand_hs_stats = [round(sum(rand_hs)/len(rand_hs),2), round(min(rand_hs),2), round(max(rand_hs),2)]

smart_hs = evaluate.hamming(target, smart_b_pred)
smart_hs_stats = [round(sum(smart_hs)/len(smart_hs),2), round(min(smart_hs),2), round(max(smart_hs),2)]


hamming_df = pd.DataFrame([hs_stats, untr_hs_stats, rand_hs_stats, smart_hs_stats], columns=['mean', 'min', 'max'], 
                            index=["DAE trained", "DAE untrained", "Random chance", "Smart random chance"])
hamming_df

In [ ]:
hs[:15]

In [ ]:
evaluate.hamming([1,1,1,1], [1,1,0,0])

In [ ]:
from sklearn.metrics import hamming_loss

In [ ]:
hamming_loss([1,1,1,1,0,0,0], [1,1,0,0,0,0,0])